# Heston Basket Options  

A basket option is a type of financial derivative where the underlying asset is a basket (group), of commodities, securities, or currencies. In some sense, it is a list of assests associated with a single option. Thus, our dataset will have a list of assest prices and correlations along with the other parameters in the normal case.   

In [1]:
import os
import time
import pathlib
import numpy as np
import pandas as pd

from heston import *
from df_helper import *
from blackscholes import *

We first load the data to generate the baskets of options using `basket_data.csv`.  

Note that the data here will be inputs to our Heston model, which will give simulated option prices for the training of our neural network.  

In [2]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/basket_data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,mean_volatility,reversion,var_of_vol,rho
0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,2.0,1.163095,0.053646,0.022956,0.047665,1.741895,0.285332,-0.557252
1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,2.0,0.997845,0.053646,0.114784,0.018427,0.765240,0.304296,-0.355317
2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,2.0,0.997223,0.053646,0.106823,0.021562,1.578755,0.553407,-0.728337
3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,2.0,1.002110,0.053646,0.110336,0.013416,2.370529,0.622139,-0.421136
4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,2.0,0.992831,0.053646,0.110336,0.011473,3.822598,0.616489,-0.572024


## Test implementation

We generate the basket options below

In [3]:
dim  = 7     # Dimension of basket (number of stocks)
num_samples = 100
inputs_array = generate_inputs_nn(df, dim, num_samples)
inputs = pd.DataFrame(inputs_array, columns = ['underlyings_price', 'rho', 'days_to_maturity', 
                        'strike', 'volatility','mean_volatility','reversion', 'vol_of_var','rate'])
print(inputs.shape)
inputs.head()

(100, 9)


,underlyings_price,rho,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate
0,"[5.456900024414063, 148.78, 16.384000244140626...","[-0.7387560637776853, -0.6348247855328666, -0....",162.0,5.8,0.002939,0.013108,2.247971,0.179029,0.002164
1,"[21.605, 13.3706298828125, 12.610999755859376,...","[-0.060916401680963655, -0.6696069122143352, -...",23.0,4.35,0.034961,0.003161,4.220833,0.631991,0.022401
2,"[15.43, 64.76169921875, 60.727001953125, 55.62...","[-0.07254262892076208, -0.1454803115726743, -0...",59.0,4.4,0.018327,0.017306,3.557934,0.100893,0.0542
3,"[15.52675048828125, 15.79, 16.66125, 11.560629...","[-0.6176187858910588, -0.13970772627889028, -0...",9.0,14.9,0.020028,0.026083,0.342213,0.288446,0.054175
4,"[14.7175, 4.975199890136719, 16.02150024414062...","[-0.5679199804147083, -0.4457859567758694, -0....",331.0,9.2,0.023719,0.035229,1.324025,0.338052,0.001639


Test the scalar case.

In [4]:
S_0   = inputs['underlyings_price'].values[0] 
v_0   = inputs['volatility'].values[0] 
r     = inputs['rate'].values[0] 
theta = inputs['mean_volatility'].values[0] 
kappa = inputs['reversion'].values[0] 
xi    = inputs['vol_of_var'].values[0] 
K     = inputs['strike'].values[0] 
rho   = inputs['rho'].values[0] 
T     = inputs['days_to_maturity'].values[0] 

scalar_result = generate_heston_paths_test(S_0, T, K, r, kappa, theta, v_0, rho, xi, 
                          100, 1000)
print(f"The scalar result is {scalar_result}")

The scalar result is 27.66047571732737


Test the vector case.

In [5]:
inputs.columns.values[2:]

array(['days_to_maturity', 'strike', 'volatility', 'mean_volatility',
       'reversion', 'vol_of_var', 'rate'], dtype=object)

In [6]:
begin = time.time()
result = generate_heston_paths_vec(inputs, num_samples, steps=100, num_sims=1000)
end = time.time()
print(f"The vector result is {result[0]}")
print(f"The shape of the result is {result.shape}")
print(f"Time taken is {end-begin} seconds")

The vector result is 30.0708477238639
The shape of the result is (100,)
Time taken is 3.5270090103149414 seconds


We can see that the first element of `generate_heston_paths_vec` is approximately the same as `generate_heston_paths`. This verified that our models are correct.

## Processing  

We now make the dataframe into the desired form for training the neural network

In [7]:
inputs['contract_price'] = result
inputs = flattenDim(7, inputs)
inputs.head()

,days_to_maturity,strike,volatility,mean_volatility,reversion,vol_of_var,rate,contract_price,Underlying_0,Underlying_1,...,Underlying_4,Underlying_5,Underlying_6,Rho_0,Rho_1,Rho_2,Rho_3,Rho_4,Rho_5,Rho_6
0,162.0,5.80,0.002939,0.013108,2.247971,0.179029,0.002164,30.070848,5.45690,148.780000,...,34.142100,5.32090,14.82250,-0.738756,-0.634825,-0.571776,-0.309349,-0.298349,-0.576349,-0.566990
1,23.0,4.35,0.034961,0.003161,4.220833,0.631991,0.022401,12.098573,21.60500,13.370630,...,16.554650,16.75125,13.99250,-0.060916,-0.669607,-0.467568,-0.310696,-0.659128,-0.616183,-0.700901
2,59.0,4.40,0.018327,0.017306,3.557934,0.100893,0.054200,34.357936,15.43000,64.761699,...,22.470750,15.00350,18.89875,-0.072543,-0.145480,-0.727849,-0.235879,-0.127984,-0.433218,-0.284496
3,9.0,14.90,0.020028,0.026083,0.342213,0.288446,0.054175,9.744849,15.52675,15.790000,...,51.603198,15.52450,4.54420,-0.617619,-0.139708,-0.623722,-0.181363,-0.144881,-0.397951,-0.358406
4,331.0,9.20,0.023719,0.035229,1.324025,0.338052,0.001639,0.988595,14.71750,4.975200,...,16.355750,16.92675,5.36400,-0.567920,-0.445786,-0.458572,-0.237322,-0.566020,-0.657473,-0.744682


In [16]:
# uncomment to save the data
# inputs.to_csv(os.path.join(parent_path, 'data/small_basket_data.csv'))